# Scraping Code

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [79]:
# Checking to see if we can scrape here

url = 'https://www.pro-football-reference.com/robots.txt'

response  = requests.get(url)
response.status_code

200

In [80]:
# Our dataframe that we need to fill

#nfl_df = pd.DataFrame(columns = ['Team', 'Year', '1st Passing %' , '2nd Passing %', '3rd Passing %', '2nd Avg Yards to Go', '3rd Avg Yards to Go', '1st 1D Conversion %', '2nd 1D Conversion %', '3rd 1D Conversion %', 'Leading % Passing', 'Tied % Passing', 'Trailing % Passing'])
nfl_df.head()

,Team,Year,1st Passing %,2nd Passing %,3rd Passing %,2nd Avg Yards to Go,3rd Avg Yards to Go,1st 1D Conversion %,2nd 1D Conversion %,3rd 1D Conversion %,Leading % Passing,Tied % Passing,Trailing % Passing


In [301]:
def pull_stats(team_acr, team, year):
    
    # Dictionary to fill per row in dataframe
    keys = ['Team', 'Year', '1st Passing %', '2nd Passing %', '3rd Passing %', '2nd Avg Yards to Go', '3rd Avg Yards to Go', '1st 1D Conversion %', '2nd 1D Conversion %', '3rd 1D Conversion %', 'Leading % Passing', 'Tied % Passing', 'Trailing % Passing']
    team_dict = dict.fromkeys(keys)

    # start on home page
    home_page = 'https://www.pro-football-reference.com/'
    #home_page = 'https://www.pro-football-reference.com/teams/buf/' #only for specific team page
    driver.get(home_page)

    # go to team page
    team_dict['Team'] = team
    search_box = driver.find_element_by_xpath("//input[@placeholder='Enter Person, Team, Section, etc']")
    search_box.send_keys(team)
    search_box.send_keys(Keys.RETURN) #hit enter
    time.sleep(10)
    
    # go to team + year page
    team_dict['Year'] = year
    team_page = driver.find_element_by_xpath("//a[@href='/teams/{}/{}.htm']".format(team_acr, year))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    driver.execute_script("arguments[0].scrollIntoView();", team_page)
    time.sleep(5)
    driver.execute_script("window.scrollBy(0, -150);")
    time.sleep(5)
    team_page.click()
    
    # offensive splits
    time.sleep(5)
    team_splits = driver.find_element_by_xpath("//div[@id='bottom_nav_container']//a[@href='/teams/{}/{}_splits.htm']".format(team_acr, year))
    driver.execute_script("arguments[0].scrollIntoView();", team_splits)
    time.sleep(5)
    driver.execute_script("window.scrollBy(0, 150);")
    team_splits.click()
    
    # soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # splits by down table
    splits_by_down_table = soup.find(id = 'div_down_splits')
    
    # First Feature
    passing_plays_1st = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['1st Passing %'] = passing_plays_1st
    
    # Second Feature
    passing_plays_2nd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['2nd Passing %'] = passing_plays_2nd

    # Third Feature
    passing_plays_3rd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['3rd Passing %'] = passing_plays_3rd

    # Fourth Feature
    yards_to_go_2nd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[2].text)
    team_dict['2nd Avg Yards to Go'] = yards_to_go_2nd

    # Fifth Feature
    yards_to_go_3rd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[3].text)
    team_dict['3rd Avg Yards to Go'] = yards_to_go_3rd
    
    # Sixth Feature
    conversion_1st = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[1].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[1].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['1st 1D Conversion %'] = conversion_1st
    
    # Seventh Feature
    conversion_2nd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[2].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[2].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['2nd 1D Conversion %'] = conversion_2nd
    
    # Eighth Feature
    conversion_3rd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[3].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[3].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['3rd 1D Conversion %'] = conversion_3rd
    
    # splits by point difference table
    splits_by_pf_table = soup.find(id = 'div_pt_diff_splits')
    
    # Nineth Feature
    leading_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['Leading % Passing'] = leading_passing_plays
    
    # Tenth Feature
    tied_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['Tied % Passing'] = tied_passing_plays
    
    # Eleventh Feature
    trailing_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['Trailing % Passing'] = trailing_passing_plays
    
    return team_dict

In [302]:
def next_pull_stats(team, year):
    # Dictionary to fill per row in dataframe
    keys = ['Team', 'Year', '1st Passing %', '2nd Passing %', '3rd Passing %', '2nd Avg Yards to Go', '3rd Avg Yards to Go', '1st 1D Conversion %', '2nd 1D Conversion %', '3rd 1D Conversion %', 'Leading % Passing', 'Tied % Passing', 'Trailing % Passing']
    team_dict = dict.fromkeys(keys)
    
    team_dict['Team'] = team
    team_dict['Year'] = year

    # Next Page
    next_page = driver.find_element_by_xpath("//div[@class='prevnext']//a[@class='button2 next']")
    next_page.click()
    time.sleep(3)
    
    # soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # splits by down table
    splits_by_down_table = soup.find(id = 'div_down_splits')
    
    # First Feature
    passing_plays_1st = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['1st Passing %'] = passing_plays_1st
    
    # Second Feature
    passing_plays_2nd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['2nd Passing %'] = passing_plays_2nd

    # Third Feature
    passing_plays_3rd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['3rd Passing %'] = passing_plays_3rd

    # Fourth Feature
    yards_to_go_2nd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[2].text)
    team_dict['2nd Avg Yards to Go'] = yards_to_go_2nd

    # Fifth Feature
    yards_to_go_3rd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[3].text)
    team_dict['3rd Avg Yards to Go'] = yards_to_go_3rd
    
    # Sixth Feature
    conversion_1st = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[1].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[1].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['1st 1D Conversion %'] = conversion_1st
    
    # Seventh Feature
    conversion_2nd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[2].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[2].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['2nd 1D Conversion %'] = conversion_2nd
    
    # Eighth Feature
    conversion_3rd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[3].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[3].text) ) / int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['3rd 1D Conversion %'] = conversion_3rd
    
    # splits by point difference table
    splits_by_pf_table = soup.find(id = 'div_pt_diff_splits')
    
    # Nineth Feature
    leading_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[1].text)
    team_dict['Leading % Passing'] = leading_passing_plays
    
    # Tenth Feature
    tied_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[2].text)
    team_dict['Tied % Passing'] = tied_passing_plays
    
    # Eleventh Feature
    trailing_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[3].text)
    team_dict['Trailing % Passing'] = trailing_passing_plays
    
    return team_dict

In [308]:
#driver = webdriver.Chrome(chromedriver)

In [309]:
#nfl_df = nfl_df.append(pull_stats('sdg', 'Los Angeles Chargers', 1994), ignore_index=True) #first year
#for year in range(1995, 2021): #rest of years
    #nfl_df = nfl_df.append(next_pull_stats('Los Angeles Chargers', year), ignore_index=True)

In [358]:
nfl_df
#nfl_df.to_csv('nfl_df.csv')

,Team,Year,1st Passing %,2nd Passing %,3rd Passing %,2nd Avg Yards to Go,3rd Avg Yards to Go,1st 1D Conversion %,2nd 1D Conversion %,3rd 1D Conversion %,Leading % Passing,Tied % Passing,Trailing % Passing,DVOA
0,Arizona Cardinals,1994,0.375000,0.509915,0.793860,8.51,8.07,0.192982,0.263456,0.337719,0.397380,0.451737,0.584369,-0.166
1,Arizona Cardinals,1995,0.436652,0.584570,0.768116,7.96,7.83,0.171946,0.299703,0.342995,0.559441,0.463415,0.584356,-0.237
2,Arizona Cardinals,1996,0.525386,0.552023,0.736842,8.04,7.24,0.185430,0.268786,0.421053,0.443966,0.513369,0.651200,-0.048
3,Arizona Cardinals,1997,0.543290,0.504323,0.669421,8.90,8.22,0.199134,0.230548,0.359504,0.477528,0.502283,0.600000,-0.138
4,Arizona Cardinals,1998,0.454936,0.505780,0.703196,7.57,6.99,0.203863,0.289017,0.347032,0.446203,0.498127,0.596567,-0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Los Angeles Chargers,2016,0.435345,0.642012,0.765000,7.99,6.78,0.217672,0.343195,0.425000,0.524752,0.531792,0.631579,NaN
847,Los Angeles Chargers,2017,0.446121,0.601190,0.808612,8.43,7.23,0.245690,0.321429,0.401914,0.498708,0.553191,0.653266,NaN
848,Los Angeles Chargers,2018,0.434590,0.553398,0.796610,7.91,7.35,0.279379,0.362460,0.384181,0.474041,0.558333,0.615183,NaN
849,Los Angeles Chargers,2019,0.498901,0.642424,0.757732,7.78,7.22,0.224176,0.339394,0.443299,0.505495,0.548780,0.658929,NaN


In [369]:
nfl_df['Team'].unique()

array(['Arizona Cardinals', 'Los Angeles Rams', 'San Francisco 49ers',
       'Seattle Seahawks', 'Denver Broncos', 'Las Vegas Raiders',
       'Kansas City Chiefs', 'Carolina Panthers', 'Tampa Bay Buccaneers',
       'New Orleans Saints', 'Atlanta Falcons', 'Tennessee Titans',
       'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars',
       'Green Bay Packers', 'Minnesota Vikings', 'Chicago Bears',
       'Detroit Lions', 'Cleveland Browns', 'Baltimore Ravens',
       'Cincinnati Bengals', 'Pittsburgh Steelers', 'Dallas Cowboys',
       'Philadelphia Eagles', 'Washington Football Team',
       'Miami Dolphins', 'New England Patriots', 'New York Jets',
       'New York Giants', 'Buffalo Bills', 'Los Angeles Chargers'],
      dtype=object)

In [372]:
len(dvoa_df['Team'].unique())

32

In [370]:
dvoa_df['Team'].replace({'SF': 'San Francisco 49ers', 'DAL': 'Dallas Cowboys', 'MIA': 'Miami Dolphins', 'PIT': 'Pittsburgh Steelers', 'GB': 'Green Bay Packers', 'SD': 'Los Angeles Chargers', 'KC': 'Kansas City Chiefs', 'DET': 'Detroit Lions', 'NO': 'New Orleans Saints', 'BUF': 'Buffalo Bills', 'DEN': 'Denver Broncos', 'LARM': 'Los Angeles Rams', 'CHI': 'Chicago Bears', 'NE': 'New England Patriots', 'NYJ': 'New York Jets', 'PHI': 'Philadelphia Eagles', 'CLE1': 'Cleveland Browns', 'MIN': 'Minnesota Vikings', 'ATL': 'Atlanta Falcons', 'IND': 'Indianapolis Colts', 'TB': 'Tampa Bay Buccaneers', 'NYG': 'New York Giants', 'LARD': 'Las Vegas Raiders', 'WAS': 'Washington Football Team', 'SEA': 'Seattle Seahawks', 'ARI': 'Arizona Cardinals', 'CIN': 'Cincinnati Bengals', 'HOIL': 'Tennessee Titans', 'OAK': 'Las Vegas Raiders', 'STL': 'Los Angeles Rams', 'JAX': 'Jacksonville Jaguars', 'CAR': 'Carolina Panthers', 'BAL': 'Baltimore Ravens', 'TEN': 'Tennessee Titans', 'CLE': 'Cleveland Browns', 'HOU': 'Houston Texans', 'LAR': 'Los Angeles Rams', 'LAC': 'Los Angeles Chargers', 'LV': 'Las Vegas Raiders'},inplace=True)


,Team,Total DVOA,Year
0,San Francisco 49ers,19.5%,1994
1,Dallas Cowboys,18.8%,1994
2,Miami Dolphins,18.5%,1994
3,Pittsburgh Steelers,13.5%,1994
4,Green Bay Packers,13.5%,1994
...,...,...,...
844,Philadelphia Eagles,-16.9%,2020
845,Cincinnati Bengals,-18.2%,2020
846,Denver Broncos,-20.0%,2020
847,New York Jets,-20.5%,2020


In [394]:
dvoa_df['DVOA'] = dvoa_df['Total DVOA'].apply(lambda x: float(x.replace('%', ''))/100)
dvoa_df

,Team,Total DVOA,Year,DVOA
0,San Francisco 49ers,19.5%,1994,0.195
1,Dallas Cowboys,18.8%,1994,0.188
2,Miami Dolphins,18.5%,1994,0.185
3,Pittsburgh Steelers,13.5%,1994,0.135
4,Green Bay Packers,13.5%,1994,0.135
...,...,...,...,...
844,Philadelphia Eagles,-16.9%,2020,-0.169
845,Cincinnati Bengals,-18.2%,2020,-0.182
846,Denver Broncos,-20.0%,2020,-0.200
847,New York Jets,-20.5%,2020,-0.205


In [398]:
final_dvoa_df = dvoa_df.drop(columns=['Total DVOA'])
final_dvoa_df

,Team,Year,DVOA
0,San Francisco 49ers,1994,0.195
1,Dallas Cowboys,1994,0.188
2,Miami Dolphins,1994,0.185
3,Pittsburgh Steelers,1994,0.135
4,Green Bay Packers,1994,0.135
...,...,...,...
844,Philadelphia Eagles,2020,-0.169
845,Cincinnati Bengals,2020,-0.182
846,Denver Broncos,2020,-0.200
847,New York Jets,2020,-0.205


In [409]:
final_nfl = nfl_df.merge(final_dvoa_df, how = 'left', on = ['Team','Year'])
final_nfl

,Team,Year,1st Passing %,2nd Passing %,3rd Passing %,2nd Avg Yards to Go,3rd Avg Yards to Go,1st 1D Conversion %,2nd 1D Conversion %,3rd 1D Conversion %,Leading % Passing,Tied % Passing,Trailing % Passing,DVOA
0,Arizona Cardinals,1994,0.375000,0.509915,0.793860,8.51,8.07,0.192982,0.263456,0.337719,0.397380,0.451737,0.584369,-0.166
1,Arizona Cardinals,1995,0.436652,0.584570,0.768116,7.96,7.83,0.171946,0.299703,0.342995,0.559441,0.463415,0.584356,-0.237
2,Arizona Cardinals,1996,0.525386,0.552023,0.736842,8.04,7.24,0.185430,0.268786,0.421053,0.443966,0.513369,0.651200,-0.048
3,Arizona Cardinals,1997,0.543290,0.504323,0.669421,8.90,8.22,0.199134,0.230548,0.359504,0.477528,0.502283,0.600000,-0.138
4,Arizona Cardinals,1998,0.454936,0.505780,0.703196,7.57,6.99,0.203863,0.289017,0.347032,0.446203,0.498127,0.596567,-0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Los Angeles Chargers,2016,0.435345,0.642012,0.765000,7.99,6.78,0.217672,0.343195,0.425000,0.524752,0.531792,0.631579,-0.033
847,Los Angeles Chargers,2017,0.446121,0.601190,0.808612,8.43,7.23,0.245690,0.321429,0.401914,0.498708,0.553191,0.653266,0.103
848,Los Angeles Chargers,2018,0.434590,0.553398,0.796610,7.91,7.35,0.279379,0.362460,0.384181,0.474041,0.558333,0.615183,0.208
849,Los Angeles Chargers,2019,0.498901,0.642424,0.757732,7.78,7.22,0.224176,0.339394,0.443299,0.505495,0.548780,0.658929,0.035


In [420]:
final_nfl.head()

,Team,Year,1st Passing %,2nd Passing %,3rd Passing %,2nd Avg Yards to Go,3rd Avg Yards to Go,1st 1D Conversion %,2nd 1D Conversion %,3rd 1D Conversion %,Leading % Passing,Tied % Passing,Trailing % Passing,DVOA
0,Arizona Cardinals,1994,0.375000,0.509915,0.793860,8.51,8.07,0.192982,0.263456,0.337719,0.397380,0.451737,0.584369,-0.166
1,Arizona Cardinals,1995,0.436652,0.584570,0.768116,7.96,7.83,0.171946,0.299703,0.342995,0.559441,0.463415,0.584356,-0.237
2,Arizona Cardinals,1996,0.525386,0.552023,0.736842,8.04,7.24,0.185430,0.268786,0.421053,0.443966,0.513369,0.651200,-0.048
3,Arizona Cardinals,1997,0.543290,0.504323,0.669421,8.90,8.22,0.199134,0.230548,0.359504,0.477528,0.502283,0.600000,-0.138
4,Arizona Cardinals,1998,0.454936,0.505780,0.703196,7.57,6.99,0.203863,0.289017,0.347032,0.446203,0.498127,0.596567,-0.062


In [428]:
final_nfl.to_csv('final_nfl.csv')

# Gathering DVOA Data

In [328]:
# dvoa_1994 = pd.read_csv('1994 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1994['Year'] = 1994
# dvoa_1994

,Team,Total DVOA,Year
0,SF,19.5%,1994
1,DAL,18.8%,1994
2,MIA,18.5%,1994
3,PIT,13.5%,1994
4,GB,13.5%,1994
5,SD,9.2%,1994
6,KC,8.1%,1994
7,DET,7.4%,1994
8,NO,7.3%,1994
9,BUF,5.0%,1994


In [329]:
# dvoa_1995 = pd.read_csv('1995 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1995['Year'] = 1995
# dvoa_1995

,Team,Total DVOA,Year
0,DAL,30.1%,1995
1,GB,20.8%,1995
2,CHI,20.5%,1995
3,SF,19.5%,1995
4,DEN,18.2%,1995
5,DET,15.7%,1995
6,ATL,14.9%,1995
7,MIA,6.4%,1995
8,SD,5.3%,1995
9,NO,3.8%,1995


In [330]:
# dvoa_1996 = pd.read_csv('1996 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1996['Year'] = 1996
# dvoa_1996

,Team,Total DVOA,Year
0,BAL,22.8%,1996
1,WAS,17.3%,1996
2,GB,16.1%,1996
3,DEN,13.6%,1996
4,KC,11.6%,1996
5,SF,10.1%,1996
6,CIN,8.4%,1996
7,JAX,7.6%,1996
8,MIA,7.2%,1996
9,PHI,7.1%,1996


In [331]:
# dvoa_1997 = pd.read_csv('1997 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1997['Year'] = 1997
# dvoa_1997

,Team,Total DVOA,Year
0,JAX,23.4%,1997
1,DEN,20.2%,1997
2,CIN,18.0%,1997
3,GB,15.9%,1997
4,KC,13.8%,1997
5,PIT,13.4%,1997
6,MIA,11.2%,1997
7,MIN,9.1%,1997
8,NE,7.5%,1997
9,SF,5.7%,1997


In [332]:
# dvoa_1998 = pd.read_csv('1998 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1998['Year'] = 1998
# dvoa_1998

,Team,Total DVOA,Year
0,DEN,35.0%,1998
1,SF,29.3%,1998
2,MIN,23.9%,1998
3,BUF,20.0%,1998
4,NYJ,17.9%,1998
5,DAL,14.9%,1998
6,TEN,12.3%,1998
7,JAX,9.9%,1998
8,ATL,7.0%,1998
9,GB,6.6%,1998


In [333]:
# dvoa_1999 = pd.read_csv('1999 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_1999['Year'] = 1999
# dvoa_1999

,Team,Total DVOA,Year
0,WAS,21.8%,1999
1,OAK,20.8%,1999
2,TEN,18.5%,1999
3,STL,18.1%,1999
4,IND,14.2%,1999
5,KC,14.1%,1999
6,BUF,12.8%,1999
7,MIN,11.9%,1999
8,JAX,11.5%,1999
9,CAR,10.0%,1999


In [334]:
# dvoa_2000 = pd.read_csv('2000 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2000['Year'] = 2000
# dvoa_2000

,Team,Total DVOA,Year
0,STL,27.0%,2000
1,IND,26.7%,2000
2,DEN,20.2%,2000
3,SF,18.8%,2000
4,MIN,18.5%,2000
5,OAK,17.4%,2000
6,KC,13.3%,2000
7,NYG,9.1%,2000
8,PIT,8.9%,2000
9,JAX,6.6%,2000


In [335]:
# dvoa_2001 = pd.read_csv('2001 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2001['Year'] = 2001
# dvoa_2001

,Team,Total DVOA,Year
0,STL,21.3%,2001
1,SF,21.2%,2001
2,OAK,18.4%,2001
3,PIT,14.9%,2001
4,KC,10.9%,2001
5,GB,8.5%,2001
6,TEN,7.6%,2001
7,IND,6.8%,2001
8,ARI,5.4%,2001
9,NYJ,4.2%,2001


In [336]:
# dvoa_2002 = pd.read_csv('2002 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2002['Year'] = 2002
# dvoa_2002

,Team,Total DVOA,Year
0,KC,35.4%,2002
1,OAK,22.8%,2002
2,SF,19.0%,2002
3,NYJ,18.4%,2002
4,DEN,17.0%,2002
5,TEN,9.0%,2002
6,SEA,7.8%,2002
7,JAX,7.6%,2002
8,NE,7.0%,2002
9,PHI,6.9%,2002


In [337]:
# dvoa_2003 = pd.read_csv('2003 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2003['Year'] = 2003
# dvoa_2003

,Team,Total DVOA,Year
0,KC,33.9%,2003
1,TEN,18.0%,2003
2,IND,17.9%,2003
3,PHI,17.6%,2003
4,SEA,16.7%,2003
5,MIN,13.7%,2003
6,GB,12.1%,2003
7,NYJ,12.0%,2003
8,DEN,9.6%,2003
9,CIN,8.3%,2003


In [338]:
# dvoa_2004 = pd.read_csv('2004 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2004['Year'] = 2004
# dvoa_2004

,Team,Total DVOA,Year
0,IND,32.2%,2004
1,KC,31.1%,2004
2,NE,24.5%,2004
3,NYJ,21.1%,2004
4,SD,17.3%,2004
5,MIN,17.0%,2004
6,GB,17.0%,2004
7,PIT,16.9%,2004
8,PHI,15.1%,2004
9,DEN,14.2%,2004


In [339]:
# dvoa_2005 = pd.read_csv('2005 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2005['Year'] = 2005
# dvoa_2005

,Team,Total DVOA,Year
0,SEA,28.9%,2005
1,DEN,27.4%,2005
2,IND,25.1%,2005
3,KC,23.9%,2005
4,CIN,23.1%,2005
5,SD,22.5%,2005
6,NE,18.0%,2005
7,PIT,12.3%,2005
8,NYG,10.4%,2005
9,WAS,8.4%,2005


In [340]:
# dvoa_2006 = pd.read_csv('2006 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2006['Year'] = 2006
# dvoa_2006

,Team,Total DVOA,Year
0,IND,29.4%,2006
1,SD,25.8%,2006
2,PHI,21.2%,2006
3,NE,14.7%,2006
4,CIN,14.4%,2006
5,DAL,12.1%,2006
6,NYG,11.8%,2006
7,NO,10.1%,2006
8,JAX,7.6%,2006
9,PIT,7.4%,2006


In [341]:
# dvoa_2007 = pd.read_csv('2007 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2007['Year'] = 2007
# dvoa_2007

,Team,Total DVOA,Year
0,NE,44.1%,2007
1,IND,23.0%,2007
2,JAX,21.9%,2007
3,DAL,17.4%,2007
4,GB,16.0%,2007
5,PHI,12.7%,2007
6,CIN,12.2%,2007
7,DEN,9.1%,2007
8,CLE,8.2%,2007
9,PIT,8.1%,2007


In [342]:
# dvoa_2008 = pd.read_csv('2008 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2008['Year'] = 2008
# dvoa_2008

,Team,Total DVOA,Year
0,NYG,19.3%,2008
1,SD,18.9%,2008
2,DEN,18.9%,2008
3,NO,16.5%,2008
4,IND,14.2%,2008
5,CAR,13.3%,2008
6,NE,12.4%,2008
7,MIA,12.2%,2008
8,ATL,8.9%,2008
9,WAS,8.4%,2008


In [343]:
# dvoa_2009 = pd.read_csv('2009 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2009['Year'] = 2009
# dvoa_2009

,Team,Total DVOA,Year
0,NE,26.7%,2009
1,NO,24.7%,2009
2,DAL,21.8%,2009
3,SD,19.8%,2009
4,GB,19.4%,2009
5,IND,17.2%,2009
6,PIT,14.7%,2009
7,MIN,13.1%,2009
8,BAL,12.9%,2009
9,HOU,10.2%,2009


In [344]:
# dvoa_2010 = pd.read_csv('2010 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2010['Year'] = 2010
# dvoa_2010

,Team,Total DVOA,Year
0,NE,42.8%,2010
1,HOU,21.6%,2010
2,PHI,18.2%,2010
3,SD,15.6%,2010
4,PIT,14.5%,2010
5,IND,13.1%,2010
6,GB,11.9%,2010
7,ATL,8.3%,2010
8,TB,8.1%,2010
9,NYG,7.3%,2010


In [345]:
# dvoa_2011 = pd.read_csv('2011 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2011['Year'] = 2011
# dvoa_2011

,Team,Total DVOA,Year
0,GB,34.2%,2011
1,NO,33.5%,2011
2,NE,32.6%,2011
3,CAR,18.5%,2011
4,SD,13.2%,2011
5,PIT,11.6%,2011
6,NYG,10.3%,2011
7,PHI,10.1%,2011
8,HOU,9.0%,2011
9,DET,7.2%,2011


In [346]:
# dvoa_2012 = pd.read_csv('2012 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2012['Year'] = 2012
# dvoa_2012

,Team,Total DVOA,Year
0,NE,31.1%,2012
1,DEN,22.7%,2012
2,GB,19.6%,2012
3,SEA,18.6%,2012
4,SF,17.4%,2012
5,WAS,16.0%,2012
6,NYG,12.9%,2012
7,DET,12.1%,2012
8,NO,12.0%,2012
9,CAR,7.6%,2012


In [347]:
# dvoa_2013 = pd.read_csv('2013 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2013['Year'] = 2013
# dvoa_2013

,Team,Total DVOA,Year
0,DEN,34.2%,2013
1,SD,23.6%,2013
2,PHI,22.6%,2013
3,NE,16.5%,2013
4,NO,16.3%,2013
5,CHI,13.4%,2013
6,SEA,10.1%,2013
7,SF,9.1%,2013
8,CAR,8.7%,2013
9,GB,8.5%,2013


In [348]:
# dvoa_2014 = pd.read_csv('2014 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2014['Year'] = 2014
# dvoa_2014

,Team,Total DVOA,Year
0,GB,25.2%,2014
1,PIT,22.9%,2014
2,DEN,20.3%,2014
3,SEA,17.4%,2014
4,DAL,17.2%,2014
5,NE,13.9%,2014
6,MIA,10.4%,2014
7,NO,10.1%,2014
8,BAL,9.8%,2014
9,SD,7.0%,2014


In [349]:
# dvoa_2015 = pd.read_csv('2015 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2015['Year'] = 2015
# dvoa_2015

,Team,Total DVOA,Year
0,SEA,19.7%,2015
1,CIN,19.4%,2015
2,PIT,17.7%,2015
3,ARI,16.2%,2015
4,NE,15.4%,2015
5,KC,13.1%,2015
6,CAR,10.5%,2015
7,NO,9.8%,2015
8,BUF,9.7%,2015
9,CHI,6.6%,2015


In [350]:
# dvoa_2016 = pd.read_csv('2016 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2016['Year'] = 2016
# dvoa_2016

,Team,Total DVOA,Year
0,ATL,25.1%,2016
1,NE,21.2%,2016
2,DAL,20.2%,2016
3,GB,16.6%,2016
4,WAS,15.5%,2016
5,NO,15.2%,2016
6,OAK,12.8%,2016
7,PIT,12.7%,2016
8,TEN,10.8%,2016
9,BUF,10.5%,2016


In [351]:
# dvoa_2017 = pd.read_csv('2017 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2017['Year'] = 2017
# dvoa_2017

,Team,Total DVOA,Year
0,NE,27.9%,2017
1,NO,22.4%,2017
2,PIT,17.5%,2017
3,KC,16.4%,2017
4,MIN,12.5%,2017
5,LAR,11.3%,2017
6,PHI,10.5%,2017
7,LAC,10.3%,2017
8,ATL,8.5%,2017
9,DAL,6.6%,2017


In [352]:
# dvoa_2018 = pd.read_csv('2018 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2018['Year'] = 2018
# dvoa_2018

,Team,Total DVOA,Year
0,KC,35.4%,2018
1,LAR,25.0%,2018
2,LAC,20.8%,2018
3,NO,16.1%,2018
4,NE,14.7%,2018
5,PIT,14.3%,2018
6,GB,11.2%,2018
7,SEA,9.1%,2018
8,ATL,8.9%,2018
9,IND,7.9%,2018


In [353]:
# dvoa_2019 = pd.read_csv('2019 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2019['Year'] = 2019
# dvoa_2019

,Team,Total DVOA,Year
0,BAL,28.2%,2019
1,DAL,24.5%,2019
2,KC,23.5%,2019
3,NO,22.0%,2019
4,SEA,17.7%,2019
5,TEN,13.0%,2019
6,SF,7.7%,2019
7,GB,6.6%,2019
8,OAK,5.4%,2019
9,MIN,4.8%,2019


In [354]:
# dvoa_2020 = pd.read_csv('2020 Team DVOA Ratings Offense.csv', usecols = [0,2])
# dvoa_2020['Year'] = 2020
# dvoa_2020

,Team,Total DVOA,Year
0,GB,29.1%,2020
1,KC,23.9%,2020
2,TB,19.8%,2020
3,TEN,18.4%,2020
4,BUF,15.6%,2020
5,SEA,13.7%,2020
6,NO,10.7%,2020
7,MIN,6.3%,2020
8,CLE,5.4%,2020
9,LAR,4.4%,2020


In [362]:
dvoa_df = pd.concat([dvoa_1994, dvoa_1995, dvoa_1996, dvoa_1997, dvoa_1998, dvoa_1999, dvoa_2000, dvoa_2001, dvoa_2002, dvoa_2003, dvoa_2004, dvoa_2005, dvoa_2006, dvoa_2007, dvoa_2008, dvoa_2009, dvoa_2010, dvoa_2011, dvoa_2012, dvoa_2013, dvoa_2014, dvoa_2015, dvoa_2016, dvoa_2017, dvoa_2018, dvoa_2019, dvoa_2020,], axis=0, ignore_index=True)
dvoa_df

,Team,Total DVOA,Year
0,SF,19.5%,1994
1,DAL,18.8%,1994
2,MIA,18.5%,1994
3,PIT,13.5%,1994
4,GB,13.5%,1994
...,...,...,...
844,PHI,-16.9%,2020
845,CIN,-18.2%,2020
846,DEN,-20.0%,2020
847,NYJ,-20.5%,2020


## Test Code

In [ ]:
# team_page = driver.find_element_by_xpath("//a[@href='/teams/crd/{}.htm']").format(year)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(10)

In [ ]:
# driver.execute_script("arguments[0].scrollIntoView();", team_page)

In [ ]:
# time.sleep(10)
# driver.execute_script("window.scrollBy(0, -150);")

In [ ]:
# time.sleep(5)
# team_page.click()

In [ ]:
# # Grab offensive splits
# time.sleep(10)
# Arizona_2016_splits = driver.find_element_by_xpath("//div[@id='bottom_nav_container']//a[@href='/teams/crd/2016_splits.htm']")
# driver.execute_script("arguments[0].scrollIntoView();", Arizona_2016_splits)

In [ ]:
# time.sleep(10)
# driver.execute_script("window.scrollBy(0, 150);")

In [ ]:
# Arizona_2016_splits.click()

In [ ]:
#soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# splits_by_down_table = soup.find(id = 'div_down_splits')

In [ ]:
# num_of_plays_1st = int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[1].text)
# num_of_plays_1st

In [ ]:
# First Feature
# passing_plays_1st = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / num_of_plays_1st
# passing_plays_1st

In [ ]:
# num_of_plays_2nd = int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[2].text)
# num_of_plays_2nd

In [ ]:
# Second Feature
# passing_plays_2nd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / num_of_plays_2nd
# passing_plays_2nd

In [ ]:
# num_of_plays_3rd = int(splits_by_down_table.find_all(attrs={"data-stat": "plays"})[3].text)
# num_of_plays_3rd

In [ ]:
# Third Feature
# passing_plays_3rd = int(splits_by_down_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / num_of_plays_3rd
# passing_plays_3rd

In [ ]:
# Fourth Feature
# yards_to_go_2nd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[2].text)
# yards_to_go_2nd

In [ ]:
# Fifth Feature
# yards_to_go_3rd = float(splits_by_down_table.find_all(attrs={"data-stat": "yds_to_go"})[3].text)
# yards_to_go_3rd

In [ ]:
# Sixth Feature
# conversion_1st = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[1].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[1].text) ) / num_of_plays_1st
# conversion_1st

In [ ]:
# Seventh Feature
# conversion_2nd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[2].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[2].text) ) / num_of_plays_2nd
# conversion_2nd

In [ ]:
# Eighth Feature
# conversion_3rd = (int(splits_by_down_table.find_all(attrs={"data-stat": "rush_first_down"})[3].text) + int(splits_by_down_table.find_all(attrs={"data-stat": "pass_first_down"})[3].text) ) / num_of_plays_3rd
# conversion_3rd

In [ ]:
# splits_by_pf_table = soup.find(id = 'div_pt_diff_splits')

In [ ]:
# Nineth Feature
# leading_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[1].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[1].text)
# leading_passing_plays

In [ ]:
# Tenth Feature
# tied_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[2].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[2].text)
# tied_passing_plays

In [ ]:
# Eleventh Feature
# trailing_passing_plays = int(splits_by_pf_table.find_all(attrs={"data-stat": "pass_att"})[3].text) / int(splits_by_pf_table.find_all(attrs={"data-stat": "plays"})[3].text)
# trailing_passing_plays

# Quit

In [313]:
#driver.quit()